In [4]:
# Dados vacinacao Covid 19 - Maranhao
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .config('spark.driver.memory', '4g')\
        .config('spark.executor.memory', '4g')\
        .master('local[2]')\
        .getOrCreate()

22/06/30 17:30:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [12]:
df = spark.read.load('db-vacinacao-ma/sample.csv', 
                     format='csv', sep=',', 
                     inferSchema='true', header='true')

In [13]:
df.printSchema()

root
 |-- document_id: string (nullable = true)
 |-- paciente_id: string (nullable = true)
 |-- paciente_idade: integer (nullable = true)
 |-- paciente_dataNascimento: timestamp (nullable = true)
 |-- paciente_enumSexoBiologico: string (nullable = true)
 |-- paciente_racaCor_codigo: integer (nullable = true)
 |-- paciente_racaCor_valor: string (nullable = true)
 |-- paciente_endereco_coIbgeMunicipio: integer (nullable = true)
 |-- paciente_endereco_coPais: integer (nullable = true)
 |-- paciente_endereco_nmMunicipio: string (nullable = true)
 |-- paciente_endereco_nmPais: string (nullable = true)
 |-- paciente_endereco_uf: string (nullable = true)
 |-- paciente_endereco_cep: integer (nullable = true)
 |-- paciente_nacionalidade_enumNacionalidade: string (nullable = true)
 |-- estabelecimento_valor: integer (nullable = true)
 |-- estabelecimento_razaoSocial: string (nullable = true)
 |-- estalecimento_noFantasia: string (nullable = true)
 |-- estabelecimento_municipio_codigo: integer (n

In [14]:
df.limit(5).toPandas()

,document_id,paciente_id,paciente_idade,paciente_dataNascimento,paciente_enumSexoBiologico,paciente_racaCor_codigo,paciente_racaCor_valor,paciente_endereco_coIbgeMunicipio,paciente_endereco_coPais,paciente_endereco_nmMunicipio,...,vacina_categoria_codigo,vacina_categoria_nome,vacina_lote,vacina_fabricante_nome,vacina_fabricante_referencia,vacina_dataAplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,sistema_origem
0,d3409961-c7bd-4a76-b090-cab593370801-i0b0,910a39296527be01599b3c867d78bf262d63ad52af7fc8...,79,1942-08-26,M,4,AMARELA,210125,10,BACABEIRA,...,2,Faixa Etária,FM2948,PFIZER,Organization/28290,2022-05-02,2º Reforço,87,COVID-19 PFIZER - COMIRNATY,Novo PNI
1,f95394b1-b1c8-4195-b760-647a1e3e1ef5-i0b0,ebaccb69468b93e30a025e2fed105734fe07c0fa82f5b0...,46,1974-10-06,F,99,SEM INFORMACAO,211130,10,SAO LUIS,...,9,Trabalhadores de Saúde,4120Z005,ASTRAZENECA/FIOCRUZ,Organization/33781055000135,2021-01-27,1ª Dose,85,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,Novo PNI
2,d352787b-93fc-4ed1-8208-84f478f0e36a-i0b0,2de3155bc97f269b04a927ca0e1f6b14f5d61cac10b7e3...,74,1947-04-06,M,3,PARDA,210990,10,SANTA INES,...,2,Faixa Etária,FF8842,PFIZER,Organization/00394544000851,2021-10-07,Reforço,87,COVID-19 PFIZER - COMIRNATY,Novo PNI
3,f9564f67-93df-46f9-a99d-439b758ae8ee-i0b0,a77405269aab8cd1191b6aedc5845ad3247010d5412006...,17,2004-07-16,F,2,PRETA,210590,10,LAGO VERDE,...,2,Faixa Etária,FM3884,PFIZER,Organization/28290,2022-03-16,2ª Dose,87,COVID-19 PFIZER - COMIRNATY,Novo PNI
4,d35505fb-1b7b-4714-991b-e3b4e8b712a7-i0b0,db91c190c2f0922712980459f98a5aa7deecadcf331feb...,34,1987-03-23,F,3,PARDA,211120,10,SAO JOSE DE RIBAMAR,...,2,Faixa Etária,219VCD292Z,ASTRAZENECA/FIOCRUZ,Organization/149,2022-02-25,Reforço,85,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,Novo PNI


In [6]:
df = df.drop("document_id", 
        "paciente_racaCor_codigo",
        "paciente_endereco_coIbgeMunicipio",
        "paciente_endereco_coPais",
        "paciente_endereco_cep",
        "estabelecimento_municipio_codigo",
        "vacina_grupoAtendimento_codigo",
        "vacina_categoria_codigo",
        "vacina_codigo")

In [17]:
df.write.format("parquet")\
    .mode("overwrite")\
    .save("processing/regs-vac-ma.parquet")

In [18]:
df = spark.read.parquet("processing/regs-vac-ma.parquet")